Train CVE description with replaced subject

In [4]:
import pickle
import numpy as np
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
from keras.callbacks import Callback
from sklearn.preprocessing import LabelEncoder

class F1ScoreCallback(Callback):
    def __init__(self, X_val, y_val):
        super(F1ScoreCallback, self).__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_f1 = 0.0
        self.best_model = None
        self.f1_scores = []

    def on_epoch_end(self, epoch, logs=None):
        y_val_pred = np.argmax(self.model.predict(self.X_val), axis=1)
        f1 = f1_score(self.y_val, y_val_pred, average='weighted')
        self.f1_scores.append(f1)
        

        if f1 > self.best_f1:
            self.best_f1 = f1
            self.best_model = self.model
            print(f"Epoch {epoch + 1} - F1 Score: {f1:.4f}")
            print("Saved best model")
            print(self.f1_scores)

with open('train_repl_subj.pickle', 'rb') as f1:
    balanced = pickle.load(f1)

with open('test_repl_subj.pickle', 'rb') as f2:
    unbalanced = pickle.load(f2)

train = np.array([item['cve_description_replaced_subject_ada_embedding'] for item in balanced if item['cwe'] != 'None'])
test = np.array([item['cwe'] for item in balanced if item['cwe'] != 'None'])
np.random.seed(42)
X_train, X_val, y_train, y_val = train_test_split(train,test,test_size=0.1,random_state=42)

X_test = np.array([item['cve_description_replaced_subject_ada_embedding'] for item in unbalanced if item['cwe'] != 'None'])
y_test = np.array([item['cwe'] for item in unbalanced if item['cwe'] != 'None'])

label_encoder_train = LabelEncoder()
y_train_encoded = label_encoder_train.fit_transform(y_train)
label_encoder_test = LabelEncoder()
y_test_encoded = label_encoder_test.fit_transform(y_test)


input_dim = X_train.shape[1]
output_dim = len(np.unique(y_train))

model = Sequential()
model.add(Dense(128, input_dim=input_dim, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

f1_callback = F1ScoreCallback(X_val, label_encoder_train.transform(y_val))

history = model.fit(X_train, y_train_encoded, epochs=40, batch_size=32, validation_data=(X_val, label_encoder_train.transform(y_val)), verbose=1, callbacks=[f1_callback])

best_model = f1_callback.best_model


# Save the best model
joblib.dump(best_model, 'CWE_classes.joblib')

# Make predictions on the test set
y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

y_pred_original = label_encoder_train.inverse_transform(y_pred)

print("Classification Report:\n", classification_report(y_test, y_pred_original, digits=4))

joblib.dump(label_encoder_train, 'label_encoder_train.joblib')

Epoch 1/40
226/226 [==============================] - 1s 3ms/step loss: 
Epoch 1 - F1 Score: 0.5212
Saved best model
[0.5211711677862818]
2026/2026 [==============================] - 11s 5ms/step - loss: 2.1243 - accuracy: 0.4452 - val_loss: 1.5971 - val_accuracy: 0.5545
Epoch 2/40
226/226 [==============================] - 1s 2ms/step loss: 1.4
Epoch 2 - F1 Score: 0.5772
Saved best model
[0.5211711677862818, 0.577220215940376]
2026/2026 [==============================] - 8s 4ms/step - loss: 1.4646 - accuracy: 0.5922 - val_loss: 1.4112 - val_accuracy: 0.6041
Epoch 3/40
226/226 [==============================] - 0s 2ms/step loss: 1.330
Epoch 3 - F1 Score: 0.6164
Saved best model
[0.5211711677862818, 0.577220215940376, 0.6164249415229962]
2026/2026 [==============================] - 9s 4ms/step - loss: 1.3304 - accuracy: 0.6254 - val_loss: 1.3325 - val_accuracy: 0.6285
Epoch 4/40
226/226 [==============================] - 0s 2ms/step loss: 1.2534 
Epoch 4 - F1 Score: 0.6273
Saved best mo

['label_encoder_train.joblib']

Inference CVE description with replaced subject

In [6]:
import pickle
import numpy as np
from sklearn.metrics import classification_report
import joblib

# Load the saved model
best_model = joblib.load('CWE_classes.joblib')

# Load the label encoder
label_encoder_train = joblib.load('label_encoder_train.joblib')

# Load the test data
with open('test_repl_subj.pickle', 'rb') as f2:
    unbalanced = pickle.load(f2)

X_test = np.array([item['cve_description_replaced_subject_ada_embedding'] for item in unbalanced if item['cwe'] != 'None'])
y_test = np.array([item['cwe'] for item in unbalanced if item['cwe'] != 'None'])

# Make predictions on the test set
y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Convert the predicted labels back to their original form
y_pred_original = label_encoder_train.inverse_transform(y_pred)

# Generate and print the classification report
print("Classification Report:\n", classification_report(y_test, y_pred_original, digits=4))


402/402 [==============================] - 1s 2ms/step
Classification Report:
               precision    recall  f1-score   support

         119     0.8127    0.4785    0.6024      1070
         120     0.3023    0.6633    0.4153       196
         125     0.7864    0.7613    0.7736       532
         134     0.4483    0.6842    0.5417        19
         190     0.4710    0.6900    0.5598       200
          20     0.4938    0.2963    0.3704       810
         200     0.8011    0.4915    0.6092       590
         203     0.6207    0.6667    0.6429        27
          22     0.8489    0.7375    0.7893       518
         269     0.2023    0.5000    0.2880       106
         276     0.1600    0.1875    0.1727        64
         287     0.4306    0.6316    0.5121       285
         295     0.6500    0.6420    0.6460        81
         306     0.3063    0.3617    0.3317        94
         312     0.1818    0.2381    0.2062        42
         319     0.4727    0.5098    0.4906        51
  